In [1]:
from IPython import embed
from dotenv import load_dotenv
from jupyter_server.auth.security import persist_config

from langchain_llm_test import ai_message
%pip install --upgrade --quiet  docx2txt langchain-community



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU langchain-text-splitters


In [12]:
%pip install langchain-chroma

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 36.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 38.5 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 36.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 16.8 MB/s eta 0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=7c6e90f77e79cca1980c5c0a87fc52c110b79718d2f5b5ba2d9a4111d2069f63
  Stored in directory: /Users/xeroman.k/Library/Caches/pip/wheels/b4/f8/a5/28e9c1524d320f4b8eefdce0e487b5c2e128dbf2ed1bb4a60b
Successfu

In [ ]:
%pip install -U langchain langchainhub --quiet


In [54]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax_docs/tax_with_table.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)

In [55]:
len(document_list)

225

In [56]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-large"
)

In [57]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding, persist_directory="./chroma", collection_name="chroma-tax")
database = Chroma(persist_directory="./chroma", collection_name="chroma-tax", embedding_function=embedding)

In [58]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

# retrieved_docs = database.similarity_search(query, 3)

In [59]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o')


In [60]:
# ai_message = llm.invoke(prompt)

In [61]:
# ai_message.content

In [62]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [63]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [64]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [66]:
ai_message = qa_chain({"query": query})

In [67]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 근로소득에 대한 소득세는 기본세율이 적용됩니다. 소득세의 기본세율은 근로소득에 대한 세율로, 이를 계산하기 위해서는 구체적인 세율 표나 기타 공제를 고려해야 합니다. 일반적으로 소득세는 누진세 구조를 가지며, 정확한 계산을 위해서는 소득 금액대별 세율을 확인해야 합니다.'}